In [ ]:
text = """
The Greek historian knew what he was talking about. The Nile River fed Egyptian civilization for hundreds of years.

The Longest River, The Nile is 4,160 miles long - the world’s longest river. It begins near the equator in Africa and flows north to the Mediterranean Sea.

In the south it churns with cataracts. A cataract is a waterfall. Near the sea the Nile branches into a delta. A delta is an area near a river’s mouth where the water deposits fine soil called silt. In the delta, the Nile divides into many streams.

The river is called the upper Nile in the south and the lower Nile in the north. 
For centuries, heavy rains in Ethiopia caused the Nile to flood every summer. 
The floods deposited rich soil along the Nile’s shores. This soil was fertile, which means it was good for growing crops. 
Unlike the Tigris and Euphrates, 
the Nile River flooded at the same time every year, so farmers could predict when to plant their crops.
"""

## Question : Which is the world's longest river ?
a) ____________________________________

b) Nile 

c) ____________________________________

d) ____________________________________



---





Nile -------------------->  Distractor generation algorithms ------> Missisipi, Amazon , Yangtze





In [ ]:
!pip install nltk==3.2.5

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

In [ ]:
def get_word_distractors(syn,word):
  distractors = []
  word = word.lower()
  ori_word = word
  if len(word.split())>0:
        word = word.replace(" ","_")
  hypernym = syn.hypernyms()
  if len(hypernym) == 0:
    return distractors
  for hyper in hypernym:
    for item in hyper.hyponyms():
        name = item.lemmas()[0].name()
        #print ("name ",name, " word",orig_word)
        if name == ori_word:
            continue
        name = name.replace("_"," ")
        name = " ".join(w.capitalize() for w in name.split())
        if name is not None and name not in distractors:
            distractors.append(name)
  return distractors

In [ ]:
def get_word_distractors_with_their_def_pos(word, pos_tag):
  defination_word_distractors = {}
  syns = wn.synsets(original_word,'n')
  for syn in syns:
    # print (syn, ": ",syn.definition(),"\n" )
    defination = syn.definition()
    synset_to_use = syn
    distractors_calculated = get_word_distractors(synset_to_use, original_word)
    if len(distractors_calculated) < 1:
      continue
    defination_word_distractors[defination] = distractors_calculated
  return defination_word_distractors

In [ ]:

original_word = "lion"
synset_to_use = wn.synsets(original_word,'n')[0]
print(synset_to_use)
distractors_calculated = get_word_distractors(synset_to_use, original_word)
print(distractors_calculated)

In [ ]:
get_word_distractors_with_their_def_pos(original_word, "n")

#Concept Net

In [ ]:
import requests
import pprint


In [ ]:
def get_distractors_conceptnet(word):
    word = word.lower()
    original_word = word
    if (len(word.split())>0):
        word = word.replace(" ","_")
    distractor_list = [] 
    url = "http://api.conceptnet.io/query?node=/c/en/%s/n&rel=/r/PartOf&start=/c/en/%s&limit=5"%(word,word)
    obj = requests.get(url).json()

    for edge in obj['edges']:
        link = edge['end']['term'] 

        url2 = "http://api.conceptnet.io/query?node=%s&rel=/r/PartOf&end=%s&limit=5"%(link,link)
        obj2 = requests.get(url2).json()
        for edge in obj2['edges']:
            word2 = edge['start']['label']
            if word2 not in distractor_list and original_word not in word2.lower():
                distractor_list.append(word2)
                   
    return distractor_list

In [ ]:
original_word = "California"
distractors = get_distractors_conceptnet(original_word)

print ("Original word: ",original_word)
print ("\nDistractors ",distractors)

# Sene2vec

In [ ]:
!pip install sense2vec==2.0.0

In [ ]:
!wget https://github.com/explosion/sense2vec/releases/download/v1.0.0/s2v_reddit_2015_md.tar.gz

In [ ]:
!tar -xvf  s2v_reddit_2015_md.tar.gz

In [ ]:
!ls s2v_old

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
# load sense2vec vectors
import spacy
from sense2vec import Sense2Vec
s2v = Sense2Vec().from_disk('s2v_old')

In [ ]:
word = "cricket"
word = word.lower()
word = word.replace(" ", "_")

print ("word ",word)

sense = s2v.get_best_sense(word)

print ("Best sense ",sense)
most_similar = s2v.most_similar(sense, n=12)


print (most_similar)

In [ ]:
from collections import OrderedDict
def sense2vec_get_words(word,s2v, use_name_entity = []):
    output = []
    word = word.lower()
    word = word.replace(" ", "_")

    sense = s2v.get_best_sense(word)
    most_similar = s2v.most_similar(sense, n=50)

    # print ("most_similar ",most_similar)

    for each_word in most_similar:
        append_word , word_name_entity = each_word[0].split("|")
        if use_name_entity:
          if word_name_entity not in use_name_entity:
            continue
        append_word = append_word.replace("_", " ").lower()
        if append_word.lower() != word:
            output.append(append_word.title())

    out = list(OrderedDict.fromkeys(output))
    return out

word = "Natural Language processing"
distractors = sense2vec_get_words(word,s2v,["NOUN"])

print ("Distractors for ",word, " : ")
print (distractors)

In [ ]:
word = "USA"
distractors = sense2vec_get_words("USA", s2v, ["GPE"])
print(distractors)

In [ ]:
!pip install strsim

In [ ]:
import string
# A function to get all the edits for a word
def edits(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz '+string.punctuation
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

print (edits("cat"))

In [ ]:
all_edits = edits(word.lower())

In [ ]:
filtered_distractors_edit_distance = [x for x in distractors if x.lower() not in all_edits]
print (filtered_distractors_edit_distance)

In [ ]:
from similarity.normalized_levenshtein import NormalizedLevenshtein
normalized_levenshtein = NormalizedLevenshtein()

In [ ]:
threshold = 0.7
filtered_distractors_edit_distance_and_levenshtein_distance =[x for x in filtered_distractors_edit_distance if normalized_levenshtein.distance(x.lower(),word.lower())>threshold]
print (filtered_distractors_edit_distance_and_levenshtein_distance)

In [ ]:
def sense2vec_get_words_correct_words(word,s2v, use_name_entity = []):
  try:
    distractors = sense2vec_get_words(word, s2v, use_name_entity)
  except Exception:
    return []
  all_edits = edits(word.lower())
  filtered_distractors_edit_distance = [x for x in distractors if x.lower() not in all_edits]
  filtered_distractors_edit_distance_and_levenshtein_distance =[x for x in filtered_distractors_edit_distance if normalized_levenshtein.distance(x.lower(),word.lower())>threshold]
  return filtered_distractors_edit_distance_and_levenshtein_distance

In [ ]:
print(sense2vec_get_words_correct_words("cricket", s2v, ["NOUN"])[:10])

True/False Questions

In [ ]:
!pip install --quiet torch==1.7.1
!pip install --quiet spacy==2.1.9
!pip install --quiet allennlp==2.1.0  allennlp-models==2.1.0
!pip install --quiet https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.1.0/en_core_web_sm-2.1.0.tar.gz

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")


# Dependecy Parser

In [ ]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging

dependency_predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/elmo-constituency-parser-2020.02.10.tar.gz")


In [ ]:
test_sentence = "The Nile River fed Egyptian civilization for hundreds of years."
test_sentence = test_sentence.rstrip('?:!.,;')
print (test_sentence)

# Predicating Dependecy Parser

In [ ]:
parser_output = dependency_predictor.predict(sentence=test_sentence)

In [ ]:
tree_string = parser_output["trees"]
print (tree_string)

In [ ]:
from nltk import tokenize
from nltk.tree import Tree


In [ ]:

tree = Tree.fromstring(tree_string)
print(tree)
tree.pretty_print()

# Getting Right most nounphrase or verbphrase

In [ ]:
# split at right most nounphrase or verbphrase

def get_flattened(t):
    sent_str_final = None
    if t is not None:
        sent_str = [" ".join(x.leaves()) for x in list(t)]
        sent_str_final = [" ".join(sent_str)]
        sent_str_final = sent_str_final[0]
    return sent_str_final

def get_right_most_VP_or_NP(parse_tree,last_NP = None,last_VP = None):
    if len(parse_tree.leaves()) == 1:
        return last_NP,last_VP
    last_subtree = parse_tree[-1]
    if last_subtree.label() == "NP":
        last_NP = last_subtree
    elif last_subtree.label() == "VP":
        last_VP = last_subtree
    
    return get_right_most_VP_or_NP(last_subtree,last_NP,last_VP)


last_nounphrase, last_verbphrase =  get_right_most_VP_or_NP(tree)
last_nounphrase_flattened = get_flattened(last_nounphrase)
last_verbphrase_flattened = get_flattened(last_verbphrase)

print ("Original Sentence: ",test_sentence)
print ("last_nounphrase: ",last_nounphrase )
print ("last_verbphrase: ",last_verbphrase)
print ("\n ")
print ("last_nounphrase: ",last_nounphrase_flattened )
print ("last_verbphrase: ",last_verbphrase_flattened)

In [ ]:
import re
def get_termination_portion(main_string, sub_string):
    combined_sub_string = sub_string.replace(" ", "")
    main_string_list = main_string.split()
    last_index = len(main_string_list)
    for i in range(last_index):
        check_string_list = main_string_list[i:]
        check_string = "".join(check_string_list)
        check_string = check_string.replace(" ", "")
        if check_string == combined_sub_string:
            return " ".join(main_string_list[:i])

    return None

In [ ]:
longest_phrase_to_use = max(last_nounphrase_flattened, last_verbphrase_flattened)
print ("Ending phrase: ", longest_phrase_to_use)

In [ ]:

longest_phrase_to_use = re.sub(r"-LRB- ", "(", longest_phrase_to_use)
longest_phrase_to_use = re.sub(r" -RRB-", ")", longest_phrase_to_use)
print ("Ending phrase: ", longest_phrase_to_use)

In [ ]:

split_sentence = get_termination_portion(test_sentence, longest_phrase_to_use)
print ("Original sentence : ",test_sentence)
print ("Original sentence after splitting at ending phrase: ",split_sentence)

In [ ]:
!pip install --quiet pytorch-pretrained-bert==0.6.2



In [ ]:
from allennlp.predictors import Predictor
from pytorch_pretrained_bert.tokenization_gpt2 import GPT2Tokenizer
from pytorch_pretrained_bert.modeling_gpt2 import GPT2LMHeadModel
import torch

SMALL_MODEL = 'gpt2'
MEDIUM_MODEL = 'https://storage.googleapis.com/allennlp/models/gpt2-345M-dump'

class Gpt2Predictor(Predictor):
    """
    The HuggingFace implementation of GPT-2 is not an AllenNLP model;
    however, our demo only expects an AllenNLP ``Predictor``. Accordingly,
    we implement a ``Predictor`` that wraps the HuggingFace GPT-2 implementation.
    """
    def __init__(self,
                 model_name: str = MEDIUM_MODEL,
                 cache_size: int = 0) -> None:
        """
        Each cache element is about 8MB, so size accordingly.
        """
        # Cache stores tuples, so default value is a tuple
        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        self.model = GPT2LMHeadModel.from_pretrained(model_name)

        # The end of text marker.
        self.END_OF_TEXT = self.tokenizer.encoder["<|endoftext|>"]


    def predict_json(self, inputs: dict) -> dict:
        previous_str = inputs["previous"]
        next_str = inputs.get("next")
        topk = inputs.get("topk", 10)

        logits = self._predict(previous_str, next_str)
        probabilities = torch.nn.functional.softmax(logits)

        best_logits, best_indices = logits.topk(topk)
        best_words = [self.tokenizer.decode([idx.item()])
                      for idx in best_indices]
        best_probabilities = probabilities[best_indices].tolist()

        return {
            "logits": best_logits.tolist(),
            "probabilities": best_probabilities,
            "words": best_words,
            "output": previous_str + (next_str or "")
        }

    def _predict(self, previous: str, next: str = None) -> torch.Tensor:

        past_logits, past = (None, None)

        # CASE 1: Previously seen input, no next
        if next is None and past is not None:
            return past_logits

        # CASE 2: Previously seen input, yes next
        elif past is not None:
            token_ids = self.tokenizer.encode(next)
        # CASE 3: Brand new input, no next
        elif next is None:
            token_ids = self.tokenizer.encode(previous)
        # CASE 4: Brand new input, yes next
        else:
            token_ids = self.tokenizer.encode(previous) + self.tokenizer.encode(next)

        inputs = torch.LongTensor([token_ids])

        logits, present = self.model(inputs, past=past)
        logits = logits[0, -1]

        key = previous if next is None else previous + next

        return logits

    def __getitem__(self, index: int) -> str:
        return self.tokenizer.decode([index])

In [ ]:
predictor = Gpt2Predictor()

In [ ]:
import string
import copy
def predict_until_punctuation(input_str,number_of_generated_sentece=2):
  if not input_str:
    raise Exception('input string required')
  output_str = copy.deepcopy(input_str)
  sentences = []
  for i in range(number_of_generated_sentece):
    output_str =  copy.deepcopy(input_str)
    while len(output_str) != 0 and output_str[-1] not in [".", "!", "?"]:
      prediction = predictor.predict_json({"previous": output_str})
      word = prediction["words"][i]
      output_str += word
      output_str = output_str.replace('"',"")
    sentences.append(output_str)
  return sentences

In [ ]:
sentence = predict_until_punctuation(split_sentence,number_of_generated_sentece=3)# this takes time needs to improve in some other alternative with hugging face

In [ ]:
sentence

In [ ]:
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer


In [ ]:
# GPT2tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
# GPT2model = TFGPT2LMHeadModel.from_pretrained("distilgpt2",pad_token_id=GPT2tokenizer.eos_token_id)
GPT2tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
GPT2model = TFGPT2LMHeadModel.from_pretrained("gpt2",pad_token_id=GPT2tokenizer.eos_token_id)

In [ ]:
input_ids = GPT2tokenizer.encode(split_sentence,return_tensors='tf')
print (input_ids)
maximum_length = len(split_sentence.split())+40

In [ ]:
# Activate top_k sampling and top_p sampling with only from 90% most likely words
sample_outputs = GPT2model.generate(
    input_ids, 
    do_sample=True, 
    max_length=maximum_length, 
    top_p=0.80, # 0.85 
    top_k=60,   #30
    repetition_penalty  = 10.0,
    num_return_sequences=3
)

In [ ]:
import nltk
nltk.download('punkt')
from nltk import tokenize
generated_sentences=[]

for i, sample_output in enumerate(sample_outputs):
    decoded_sentence = GPT2tokenizer.decode(sample_output, skip_special_tokens=True)
    print(decoded_sentence,"\n")
    # final_sentence = decoded_sentence
    final_sentence = tokenize.sent_tokenize(decoded_sentence)[0]
    generated_sentences.append(final_sentence)
    print (i,": ",final_sentence)

In [ ]:
!pip install --quiet sentence-transformers==1.2.0

In [ ]:
from sentence_transformers import SentenceTransformer

model_BERT = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
import scipy
# sort by sentence by disimiairity
def sort_by_similarity(original_sentence, generated_sentences_list):
    # Each sentence is encoded as a 1-D vector with 768 columns
    sentence_embeddings = model_BERT.encode(generated_sentences_list)

    queries = [original_sentence]
    query_embeddings = model_BERT.encode(queries)
    # Find the top sentences of the corpus for each query sentence based on cosine similarity
    number_top_matches = len(generated_sentences_list)

    dissimilar_sentences = []

    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings, "cosine")[0]

        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: x[1])


        for idx, distance in reversed(results[0:number_top_matches]):
            score = 1-distance
            if score < 0.9:
                dissimilar_sentences.append(generated_sentences_list[idx].strip())
           
    sorted_dissimilar_sentences = sorted(dissimilar_sentences, key=len)
    
    return sorted_dissimilar_sentences[:3]

In [ ]:
print(test_sentence)
print("Wrong Sentences:")
for i in sort_by_similarity(test_sentence,sentence):
  print(" => ", i)

In [ ]:
print(test_sentence)
print("Wrong Sentences :")
for i in sort_by_similarity(test_sentence, generated_sentences):
  print(" => ", i)

In [ ]:
!pip install --quiet contractions==0.0.49

In [ ]:
from nltk import word_tokenize,pos_tag
from nltk.corpus import wordnet
import contractions

nltk.download('averaged_perceptron_tagger')
class ChangeNegationReplacer(object):

  def prepose_sent(self, text):
    expanded_words = []    
    for word in text.split():
      # using contractions.fix to expand the shotened words
      expanded_words.append(contractions.fix(word))
    return " ".join(expanded_words)

  def replace(self, word, pos=None):
    """ Returns the antonym of a word, but only if there is no ambiguity.
      >>> replacer = AntonymReplacer()
      >>> replacer.replace('good')
      >>> replacer.replace('uglify')
      'beautify'
      >>> replacer.replace('beautify')
      'uglify'
		"""
    
    antonyms = set()
    if pos:
      pos = "a" if pos[0].lower() == "j" else pos[0].lower()
    for syn in wordnet.synsets(word, pos=pos):
      for lemma in syn.lemmas():
        if lemma.antonyms():
          for antonym in lemma.antonyms():
            antonyms.add(antonym.name())

    if len(antonyms) == 1:
      return antonyms.pop()
    else:
      return None
  def replace_negations(self, text: str, isPos: bool=False):
    contractions.fix(text)
    sent = word_tokenize(self.prepose_sent(text))
    if isPos:
      sent = pos_tag(sent)
    i, l = 0, len(sent)
    words = []
    while i < l:
      if isPos:
        word, pos = sent[i]
      else:
        word = sent[i]
      if word == 'not' and i+1 < l:
        if isPos:
          next_word, pos = sent[i+1]
        else:
          next_word = sent[i+1] 
        if isPos:
          ant = self.replace(next_word,pos)
        else:
          ant = self.replace(next_word)
        if ant:
          words.append(ant)
          i += 2
          continue
      words.append(word)
      i += 1
    return " ".join(words)

In [ ]:
text = "The amount of matter in a chemical reaction doesn't change"

In [ ]:
print(ChangeNegationReplacer().replace_negations(text,isPos=True))

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

In [ ]:
tokens = nltk.word_tokenize(text)

In [ ]:
tagged = nltk.pos_tag(tokens)

In [ ]:
tagged

In [ ]:
doc = nlp(text)

In [ ]:
sposs = {}
for sent in doc.sents:
  # We are going to prepare the dictionary of parts-of-speech as the key and value is a list of words:
  # {part-of-speech: [word1, word2]}
  # We are basically grouping the words based on the parts-of-speech
  poss = {}
  sposs[sent.text] = poss
  for word in sent:
    if word.is_space or word.is_punct or word.is_space:
      continue 
    tag = word.tag_
    if tag not in poss:
      poss[tag] = []
    poss[tag].append(word.text)

In [ ]:
!python -m textblob.download_corpora


In [ ]:
from textblob import TextBlob
ww2b = TextBlob(text)
sposs = {}
for sentence in ww2b.sentences:
  poss = {}
  sposs[sentence.string] = poss;
  for t in sentence.tags:
      tag = t[1]
      if tag not in poss:
          poss[tag] = []
      poss[tag].append(t[0])

In [ ]:
import random
import re

# Create the blank in string
def replaceIC(word, sentence):
  try:
      insensitive_hippo = re.compile(re.escape(word), re.IGNORECASE)
      return insensitive_hippo.sub('__________________', sentence)
  except Exception:
    return sentence.replace(word , '__________________')

def removeWord(sentence, poss):
    words = None
    if 'NNP' in poss:
        words = poss['NNP']
    elif 'NN' in poss:
        words = poss['NN']
    else:
        print("NN and NNP not found")
        return (None, sentence, None)
    if len(words) > 0:
        word = random.choice(words)
        replaced = replaceIC(word, sentence)
        return (word, sentence, replaced)
    else:
        print("words are empty")
        return (None, sentence, None)

In [ ]:
# Iterate over the sentenses
# Spacy based
for sentence in sposs.keys():
    poss = sposs[sentence]
    (word, osentence, replaced) = removeWord(sentence, poss)
    if replaced is None:
        print ("Founded none for ")
        print(sentence)
    else:
        print(replaced)
        print ("\n===============")
        print(word)
        print ("===============")
        print("\n")

In [ ]:
# Iterate over the sentenses
# Textblob based 
for sentence in sposs.keys():
    poss = sposs[sentence]
    (word, osentence, replaced) = removeWord(sentence, poss)
    if replaced is None:
        print ("Founded none for ")
        print(sentence)
    else:
        print(replaced)
        print ("\n===============")
        print(word)
        print ("===============")
        print("\n")

In [ ]:
# https://drive.google.com/file/d/148infzmx4C8HD3ribjei6hBO9vdrr2Tp/view?usp=sharing # base
# https://drive.google.com/file/d/1I1yFHH522XTvtquGx_IoATEqC-jF9o7r/view?usp=sharing # large

In [ ]:
!pip install gdown --quiet
!pip install patool --quiet
!pip install pyunpack --quiet

In [ ]:
!gdown --id 148infzmx4C8HD3ribjei6hBO9vdrr2Tp



In [ ]:
import os
from pyunpack import Archive
bert_wsd_pytorch = "bert_large-batch_size=128-lr=2e-5-max_gloss=6.rar"
extracted_folder = bert_wsd_pytorch.replace(".rar","")
# !rm -r "/content/bert_large-batch_size=128-lr=2e-5-max_gloss=6"
if not os.path.isdir(extracted_folder):
  # os.makedirs(extracted_folder)
  Archive(bert_wsd_pytorch).extractall(".")
else:
  print (extracted_folder," is extracted already")

In [ ]:
!pip install --quiet transformers==2.9.0
!pip install --quiet nltk==3.4.5
!pip install --quiet gradio==1.4.2

In [ ]:
import torch
import math
from transformers import BertModel, BertConfig, BertPreTrainedModel, BertTokenizer

class BertWSD(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)

        self.bert = BertModel(config)
        self.dropout = torch.nn.Dropout(config.hidden_dropout_prob)

        self.ranking_linear = torch.nn.Linear(config.hidden_size, 1)

        self.init_weights()


# def _forward(args, model, batch):
#     batch = tuple(t.to(args.device) for t in batch)
#     outputs = model.bert(input_ids=batch[0], attention_mask=batch[1], token_type_ids=batch[2])

#     return model.dropout(outputs[1])
    

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_dir = "bert_large-batch_size=128-lr=2e-5-max_gloss=6"


model = BertWSD.from_pretrained(model_dir)
tokenizer = BertTokenizer.from_pretrained(model_dir)
# add new special token
if '[TGT]' not in tokenizer.additional_special_tokens:
    tokenizer.add_special_tokens({'additional_special_tokens': ['[TGT]']})
    assert '[TGT]' in tokenizer.additional_special_tokens
    model.resize_token_embeddings(len(tokenizer))
    
model.to(DEVICE)
model.eval()

In [ ]:
import csv
import os
from collections import namedtuple

import re
import torch
from tabulate import tabulate
from torch.nn.functional import softmax
from tqdm import tqdm
from transformers import BertTokenizer
import time
from transformers import T5ForConditionalGeneration,T5Tokenizer

import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

import torch
from tqdm import tqdm

GlossSelectionRecord = namedtuple("GlossSelectionRecord", ["guid", "sentence", "sense_keys", "glosses", "targets"])
BertInput = namedtuple("BertInput", ["input_ids", "input_mask", "segment_ids", "label_id"])



def _create_features_from_records(records, max_seq_length, tokenizer, cls_token_at_end=False, pad_on_left=False,
                                  cls_token='[CLS]', sep_token='[SEP]', pad_token=0,
                                  sequence_a_segment_id=0, sequence_b_segment_id=1,
                                  cls_token_segment_id=1, pad_token_segment_id=0,
                                  mask_padding_with_zero=True, disable_progress_bar=False):
    """ Convert records to list of features. Each feature is a list of sub-features where the first element is
        always the feature created from context-gloss pair while the rest of the elements are features created from
        context-example pairs (if available)
        `cls_token_at_end` define the location of the CLS token:
            - False (Default, BERT/XLM pattern): [CLS] + A + [SEP] + B + [SEP]
            - True (XLNet/GPT pattern): A + [SEP] + B + [SEP] + [CLS]
        `cls_token_segment_id` define the segment id associated to the CLS token (0 for BERT, 2 for XLNet)
    """
    features = []
    for record in tqdm(records, disable=disable_progress_bar):
        tokens_a = tokenizer.tokenize(record.sentence)

        sequences = [(gloss, 1 if i in record.targets else 0) for i, gloss in enumerate(record.glosses)]

        pairs = []
        for seq, label in sequences:
            tokens_b = tokenizer.tokenize(seq)

            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)

            # The convention in BERT is:
            # (a) For sequence pairs:
            #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
            #  type_ids:   0   0  0    0    0     0       0   0   1  1  1  1   1   1
            #
            # Where "type_ids" are used to indicate whether this is the first
            # sequence or the second sequence. The embedding vectors for `type=0` and
            # `type=1` were learned during pre-training and are added to the wordpiece
            # embedding vector (and position vector). This is not *strictly* necessary
            # since the [SEP] token unambiguously separates the sequences, but it makes
            # it easier for the model to learn the concept of sequences.
            #
            # For classification tasks, the first vector (corresponding to [CLS]) is
            # used as as the "sentence vector". Note that this only makes sense because
            # the entire model is fine-tuned.
            tokens = tokens_a + [sep_token]
            segment_ids = [sequence_a_segment_id] * len(tokens)

            tokens += tokens_b + [sep_token]
            segment_ids += [sequence_b_segment_id] * (len(tokens_b) + 1)

            if cls_token_at_end:
                tokens = tokens + [cls_token]
                segment_ids = segment_ids + [cls_token_segment_id]
            else:
                tokens = [cls_token] + tokens
                segment_ids = [cls_token_segment_id] + segment_ids

            input_ids = tokenizer.convert_tokens_to_ids(tokens)

            # The mask has 1 for real tokens and 0 for padding tokens. Only real
            # tokens are attended to.
            input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

            # Zero-pad up to the sequence length.
            padding_length = max_seq_length - len(input_ids)
            if pad_on_left:
                input_ids = ([pad_token] * padding_length) + input_ids
                input_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + input_mask
                segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids
            else:
                input_ids = input_ids + ([pad_token] * padding_length)
                input_mask = input_mask + ([0 if mask_padding_with_zero else 1] * padding_length)
                segment_ids = segment_ids + ([pad_token_segment_id] * padding_length)

            assert len(input_ids) == max_seq_length
            assert len(input_mask) == max_seq_length
            assert len(segment_ids) == max_seq_length

            pairs.append(
                BertInput(input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids, label_id=label)
            )

        features.append(pairs)

    return features


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()
            
            


question_model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_squad_v1')
question_tokenizer = T5Tokenizer.from_pretrained('t5-base')

MAX_SEQ_LENGTH = 128

def get_sense(sent):

  re_result = re.search(r"\[TGT\](.*)\[TGT\]", sent)
  if re_result is None:
      print("\nIncorrect input format. Please try again.")

  ambiguous_word = re_result.group(1).strip()




  results = dict()

  wn_pos = wn.NOUN
  for i, synset in enumerate(set(wn.synsets(ambiguous_word, pos=wn_pos))):
      results[synset] =  synset.definition()

  if len(results) ==0:
    return (None,None,ambiguous_word)

  # print (results)
  sense_keys=[]
  definitions=[]
  for sense_key, definition in results.items():
      sense_keys.append(sense_key)
      definitions.append(definition)


  record = GlossSelectionRecord("test", sent, sense_keys, definitions, [-1])

  features = _create_features_from_records([record], MAX_SEQ_LENGTH, tokenizer,
                                            cls_token=tokenizer.cls_token,
                                            sep_token=tokenizer.sep_token,
                                            cls_token_segment_id=1,
                                            pad_token_segment_id=0,
                                            disable_progress_bar=True)[0]

  with torch.no_grad():
      logits = torch.zeros(len(definitions), dtype=torch.double).to(DEVICE)
      for i, bert_input in tqdm(list(enumerate(features)), desc="Progress"):
          logits[i] = model.ranking_linear(
              model.bert(
                  input_ids=torch.tensor(bert_input.input_ids, dtype=torch.long).unsqueeze(0).to(DEVICE),
                  attention_mask=torch.tensor(bert_input.input_mask, dtype=torch.long).unsqueeze(0).to(DEVICE),
                  token_type_ids=torch.tensor(bert_input.segment_ids, dtype=torch.long).unsqueeze(0).to(DEVICE)
              )[1]
          )
      scores = softmax(logits, dim=0)

      preds = (sorted(zip(sense_keys, definitions, scores), key=lambda x: x[-1], reverse=True))


  # print (preds)
  sense = preds[0][0]
  meaning = preds[0][1]
  return (sense,meaning,ambiguous_word)


# Distractors from Wordnet
def get_distractors_wordnet(syn,word):
    distractors=[]
    word= word.lower()
    orig_word = word
    if len(word.split())>0:
        word = word.replace(" ","_")
    hypernym = syn.hypernyms()
    if len(hypernym) == 0: 
        return distractors
    for item in hypernym[0].hyponyms():
        name = item.lemmas()[0].name()
        #print ("name ",name, " word",orig_word)
        if name == orig_word:
            continue
        name = name.replace("_"," ")
        name = " ".join(w.capitalize() for w in name.split())
        if name is not None and name not in distractors:
            distractors.append(name)
    return distractors


def get_question(sentence,answer):
  text = "context: {} answer: {} </s>".format(sentence,answer)
  max_len = 256
  encoding = question_tokenizer.encode_plus(text,max_length=max_len, pad_to_max_length=True, return_tensors="pt")

  input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]

  outs = question_model.generate(input_ids=input_ids,
                                  attention_mask=attention_mask,
                                  early_stopping=True,
                                  num_beams=5,
                                  num_return_sequences=1,
                                  no_repeat_ngram_size=2,
                                  max_length=200)


  dec = [question_tokenizer.decode(ids) for ids in outs]


  Question = dec[0].replace("question:","")
  Question= Question.strip()
  return Question

In [ ]:
import re
def getMCQs(sent):
  sentence_for_bert = sent.replace("**"," [TGT] ")
  sentence_for_bert = " ".join(sentence_for_bert.split())
  print("sentence_for_bert",sentence_for_bert)
  # try:
  sense, meaning, answer = get_sense(sentence_for_bert)
  if sense is not None:
    distractors = get_distractors_wordnet(sense,answer)
  else:
    # get distractors from sense2vec
    focus_word = sent.split("**")[1::2][0]
    senten = sent.replace("**"," ")
    doc = nlp(senten)
    tag = None
    for tok in doc:
      if tok.text == focus_word:
        tag = tok.pos_
    distractors = sense2vec_get_words_correct_words(focus_word, s2v, ["NOUN"])
  if not distractors:
    distractors = ["Word not found in Wordnet. So unable to extract distractors."]
  sentence_for_T5 = sent.replace("**"," ")
  sentence_for_T5 = " ".join(sentence_for_T5.split()) 
  ques = get_question(sentence_for_T5,answer)
  return ques,answer,distractors,meaning

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:

sentence = "Its language constructs as well as its **object-oriented** approach aim to help programmers write clear, logical code for small and large-scale projects."


question,answer,distractors,meaning = getMCQs(sentence)
print (question)
print (answer)
print (distractors)
print (meaning)

In [ ]:
doc = nlp(sentence)

In [ ]:
for token in doc:
  if token.text == "Cricket":
    print(token.pos_)
    print (token.text, token.tag_, token.pos_, spacy.explain(token.tag_))

In [ ]:
 "Mark's favourite game is **Cricket**.".split("**")[1::2][0]